![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response

# Setup

In [4]:
!pip install langchain

In [5]:
!pip install langchain_ibm

  Using cached langchain_ibm-0.3.10-py3-none-any.whl.metadata (5.2 kB)
  Using cached ibm_watsonx_ai-1.3.13-py3-none-any.whl.metadata (6.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.9 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.14.1-py3-none-any.whl size=77230 sha256=3c47ffcc9804cb28a53dc555351615ddb0764c887060e2ce706cfd17b0ba9b0f
  Stored in directory: /root/.cache/pip/wheels/17/b1/7b/2e3c0f2164c99c9e574fe42e60efdd5701e62743b4aef74f72
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.14.1-

In [7]:
!pip install ibm_watsonx_ai

In [8]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 9.2 MB/s eta 0:00:00


In [10]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [11]:
!pip install langchain_tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.6 MB/s eta 0:00:00


In [12]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [13]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

Please enter your api key (hit enter): ··········


# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [14]:
model_id = "meta-llama/llama-3-3-70b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [15]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [16]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [17]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
#add code for RAG

In [21]:
!pip install openai faiss-cpu pandas tiktoken langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.1 MB/s eta 0:00:00


In [22]:
# Imports
import pandas as pd
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from IPython.display import display
import ipywidgets as widgets

FileUpload(value={}, accept='.csv', description='Upload')

FileUpload(value={}, accept='.csv', description='Upload')

In [35]:
!pip install langchain_community

In [29]:
import pandas as pd

In [57]:
# Load CSV directly
df = pd.read_csv('/content/test.csv')
print("CSV Loaded Successfully. Preview:")
display(df.head())

# Function to create RAG chain from DataFrame
def process_and_create_rag_from_df(df):
    # Convert DataFrame to text
    text_data = df.to_csv(index=False)

    # Split into chunks using a token-aware splitter
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    import tiktoken

    # Use a RecursiveCharacterTextSplitter with tiktoken tokenizer
    # Set chunk_size well below the OpenAI token limit per request
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Reduced chunk size significantly
        chunk_overlap=200, # Adjust overlap as needed
        length_function=lambda text: len(tiktoken.encoding_for_model("text-embedding-ada-002").encode(text)), # Use token count as length
        separators=["\n\n", "\n", " ", ""] # Common separators
    )
    texts = splitter.split_text(text_data)

    # Create vector store
    # Ensure your OpenAI API key is correctly set as an environment variable or passed here
    # The API key is already present here, but it also needs to be passed to the OpenAI LLM
    embeddings = OpenAIEmbeddings(openai_api_key='YourKEY')
    vectorstore = FAISS.from_texts(texts, embedding=embeddings)

    # Create QA chain
    retriever = vectorstore.as_retriever()
    # Pass the openai_api_key to the OpenAI LLM as well
    qa = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0, openai_api_key='YOURKEY'), retriever=retriever)
    return qa

# Create the RAG agent
qa = process_and_create_rag_from_df(df)

# UI for questions
query_box = widgets.Text(
    placeholder='Ask a question based on your CSV...',
    description='Query:',
    disabled=False
)

output = widgets.Output()

def on_query_submit(change):
    output.clear_output()
    with output:
        response = qa.run(query_box.value)
        print("Answer:", response)

query_box.on_submit(on_query_submit)

display(query_box, output)

CSV Loaded Successfully. Preview:


,name,calories
0,Cornstarch,381
1,"Nuts, pecans",691
2,"Eggplant, raw",25
3,"Teff, uncooked",367
4,"Sherbet, orange",144


Text(value='', description='Query:', placeholder='Ask a question based on your CSV...')

Output()

In [ ]:


def get_schema_model(original_json_schema):
    # Create a pydantic base model class from the tool's JSON schema
    from datamodel_code_generator import DataModelType, PythonVersion
    from datamodel_code_generator.model import get_data_model_types
    from datamodel_code_generator.parser.jsonschema import JsonSchemaParser
    from typing import Optional
    from pydantic import BaseModel, Field, constr
    import json

    json_schema = json.dumps(original_json_schema)

    data_model_types = get_data_model_types(
        DataModelType.PydanticV2BaseModel,
        target_python_version=PythonVersion.PY_311
    )

    # Returns the python class code as a string
    parser = JsonSchemaParser(
        json_schema,
        data_model_type=data_model_types.data_model,
        data_model_root_type=data_model_types.root_model,
        data_model_field_type=data_model_types.field_model,
        data_type_manager_type=data_model_types.data_type_manager,
        dump_resolve_reference_action=data_model_types.dump_resolve_reference_action,
    )

    model_code = parser.parse()

    full_code = model_code
    namespace = {
        "Field": Field,
        "constr": constr,
        "Optional": Optional
    }
    value = exec(full_code, namespace)
    value = exec("Model.model_rebuild()", namespace)
    pydantic_model = namespace['Model']
    return pydantic_model

def get_remote_tool_descriptions():
    remote_tool_descriptions = {}
    remote_tool_schemas = {}
    import requests

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f'Bearer {get_bearer_token()}'
    }

    tool_url = "https://private.api.eu-de.dataplatform.cloud.ibm.com"

    remote_tools_response = requests.get(f'{tool_url}/wx/v1-beta/utility_agent_tools', headers = headers)
    remote_tools = remote_tools_response.json()

    for resource in remote_tools["resources"]:
        tool_name = resource["name"]
        tool_description = resource["description"]
        tool_schema = resource.get("input_schema")
        remote_tool_descriptions[tool_name] = tool_description
        if (tool_schema):
            remote_tool_schemas[tool_name] = get_schema_model(tool_schema)

    return remote_tool_descriptions, remote_tool_schemas

tool_descriptions, tool_schemas = get_remote_tool_descriptions()

def create_remote_tool(tool_name, context):
    from langchain_core.tools import StructuredTool
    from langchain_core.tools import Tool
    import requests

    def call_tool( tool_input ):
        body = {
            "tool_name": tool_name,
            "input": tool_input
        }

        headers  = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f'Bearer {get_bearer_token()}'
        }

        tool_url = "https://private.api.eu-de.dataplatform.cloud.ibm.com"

        tool_response = requests.post(f'{tool_url}/wx/v1-beta/utility_agent_tools/run', headers = headers, json = body)

        if (tool_response.status_code > 400):
            raise Exception(f'Error calling remote tool: {tool_response.json()}' )

        tool_output = tool_response.json()
        return tool_response.json().get("output")

    def call_tool_structured(**tool_input):
        return call_tool(tool_input)

    def call_tool_unstructured(tool_input):
        return call_tool(tool_input)

    remote_tool_schema = tool_schemas.get(tool_name)

    if (remote_tool_schema):
        tool = StructuredTool(
            name=tool_name,
            description = tool_descriptions[tool_name],
            func=call_tool_structured,
            args_schema=remote_tool_schema
        )
        return tool



    tool = Tool(
        name=tool_name,
        description = tool_descriptions[tool_name],
        func=call_tool_unstructured
    )
    return tool


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        function_name = tree.body[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)

    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=get_schema_model(tool_schema)
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    top_k_results = 5
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=top_k_results))
    tools.append(wikipedia)
    max_results = 10
    search = DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(max_results=max_results))
    tools.append(search)

    tools.append(create_remote_tool("GoogleSearch", context))
    tools.append(create_remote_tool("Weather", context))
    tools.append(create_remote_tool("WebCrawler", context))

    return tools

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """
# Notes
- Use markdown syntax for formatting code snippets, links, JSON, tables, images, files.
- Any HTML tags must be wrapped in block quotes, for example ```<html>```.
- When returning code blocks, specify language.
- Sometimes, things don't go as planned. Tools may not provide useful information on the first few tries. You should always try a few different approaches before declaring the problem unsolvable.
- When the tool doesn't give you what you were asking for, you must either use another tool or a different tool input.
- When using search engines, you try different formulations of the query, possibly even in a different language.
- You cannot do complex calculations, computations, or data manipulations without using tools.
- If you need to call a tool to compute something, always call it instead of saying you will call it.

If a tool returns an IMAGE in the result, you must include it in your answer as Markdown.

Example:

Tool result: IMAGE(https://api.eu-de.dataplatform.cloud.ibm.com/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)
Markdown to return to user: ![Generated image](https://api.eu-de.dataplatform.cloud.ibm.com/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)

You are a helpful assistant that uses tools to answer questions in detail.
When greeted, say \"Hi, I am watsonx.ai agent. How can I help you?\"
You are a \"highly knowledgeable and helpful AI nutritionist.\"
Provide \"personalized nutrition guidance based on my specific needs and goals.\"
Understand user inputs via text, voice, or image like food photos, grocery labels.
\"Please ask me clarifying questions to understand my dietary preferences, health conditions, and fitness goals.\"
After understanding my requirements, generate a detailed and tailored nutrition plan, including meal suggestions, recipes, and tips for healthy eating.\"
Generate personalized meal plans based on health goals, medical conditions, fitness routines, and preferences.
Offer contextual explanations like “Why is this food better?”.
Adapt suggestions dynamically with continuous feedback"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  